In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix
from skimage.feature import greycomatrix, greycoprops
import matplotlib.pyplot as plt
from scipy import signal as sg
import itertools
import numpy as np 
import cv2
import os
import pandas as pd

1. Extracting features in a series of windows of size w, each centered on a pixel (i,j)
    - The value of the resulting statistical measure are assigned to the position (i,j) in the new pixel


2. Generating of Texture descriptors using both
    - GLCM numeric features
    - and Law's energy feature

In [2]:
# laws texture

def laws_texture(gray_image):
    (rows, cols) = gray_image.shape[:2]
    smooth_kernel = (1/25)*np.ones((5,5))
    gray_smooth = sg.convolve(gray_image, smooth_kernel, 'same')
    gray_processed = np.abs(gray_image - gray_smooth)
    
    filter_vectors = np.array([[1,4,6,4,1],
                              [-1,-2,0,2,1],
                              [-1,0,2,0,1],
                              [1,-4,6,-4,1]])
    filters = []
    for i in range(4):
        for j in range(4):
            filters.append(np.matmul(filter_vectors[i][:].reshape(5,1),
                               filter_vectors[j][:].reshape(1,5)))
    
    conv_maps = np.zeros((rows, cols, 16))
    for i in range(len(filters)):
        conv_maps[:,:,i] = sg.convolve(gray_processed,
                                      filters[i], 'same')
    
    # texture map 계산
    """
    L5E5 / E5L5
    L5S5 / S5L5
    L5R5 / R5L5
    E5R5 / R5E5
    E5S5 / S5E5
    S5R5 / R5S5
    S5S5
    E5E5
    R5R5
    
    추가가능..?
    """
    texture_maps = list()
    texture_maps.append((conv_maps[:,:,1]+conv_maps[:,:,4])//2)
    texture_maps.append((conv_maps[:,:,2]+conv_maps[:,:,8])//2)
    texture_maps.append((conv_maps[:,:,3]+conv_maps[:,:,12])//2)
    texture_maps.append((conv_maps[:,:,7]+conv_maps[:,:,13])//2)
    texture_maps.append((conv_maps[:,:,6]+conv_maps[:,:,9])//2)
    texture_maps.append((conv_maps[:,:,11]+conv_maps[:,:,14])//2)
    texture_maps.append((conv_maps[:,:,10]))
    texture_maps.append((conv_maps[:,:,5]))
    texture_maps.append((conv_maps[:,:,15]))
    texture_maps.append((conv_maps[:,:,0])) # L5L5(use to norm TEM)
    
    # law's texture energy
    TEM = list()
    for i in range(9):
        TEM.append(np.abs(texture_maps[i]).sum() /
                  np.abs(texture_maps[9]).sum())
    return TEM

In [3]:
# feature extraction
train_dir = './train/'
test_dir = './test/'
classes = ['brick', 'grass','ground','water','wood']

X_train = [] # train data feature 저장 list
Y_train = [] # train data label 저장

PATCH_SIZES = [32]
NUM_PATCHES = 10
glcm_props = ['contrast', 'dissimilarity','homogeneity','energy','correlation','ASM']

np.random.seed(22)
for idx, texture_name in enumerate(classes):
    image_dir = os.path.join(train_dir, texture_name)
    print(len(os.listdir(image_dir)))
    for image_name in os.listdir(image_dir):
        image = cv2.imread(os.path.join(image_dir, image_name))
        image_s = cv2.resize(image, (100,100), interpolation=cv2.INTER_LINEAR)
        for PATCH_SIZE in PATCH_SIZES:
#             print(PATCH_SIZE)
            for _ in range(NUM_PATCHES):
                h = np.random.randint(100-PATCH_SIZE)
                w = np.random.randint(100-PATCH_SIZE)

                image_p = image_s[h:h+PATCH_SIZE, w:w+PATCH_SIZE] # patch 자르기
                image_p_gray = cv2.cvtColor(image_p, cv2.COLOR_BGR2GRAY) # 흑백이미지 변환
                image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # hsv이미지 변환
                glcm = greycomatrix(image_p_gray, distances=[1], angles=[0], levels=256,
                                   symmetric=False, normed=True)
                props = list()
                for prop in glcm_props:
                    props.append(greycoprops(glcm, f'{prop}')[0,0])
                
                X_train.append(props + laws_texture(image_p_gray))
                Y_train.append(idx)
                
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print('train data:', X_train.shape)
print('train label:', Y_train.shape)

10
10
10
10
10
train data: (500, 15)
train label: (500,)


In [4]:
train_data = np.append(X_train, Y_train.reshape(-1,1), axis=1)

In [5]:
X_train_df = pd.DataFrame(train_data, columns=['contrast', 'dissimilarity','homogeneity',
                                            'energy','correlation','ASM']+[f'TEM{i}' for i in range(9)]+['label'])

In [6]:
X_train_df.shape

(500, 16)

In [7]:
X_train_df.head()

,contrast,dissimilarity,homogeneity,energy,correlation,ASM,TEM0,TEM1,TEM2,TEM3,TEM4,TEM5,TEM6,TEM7,TEM8,label
0,736.800403,19.854839,0.053637,0.034540,0.498253,0.001193,0.068987,0.132332,0.129375,0.037056,0.014297,0.032857,0.020925,0.021652,0.125093,0.0
1,657.841734,19.920363,0.052420,0.034274,0.345830,0.001175,0.060961,0.132345,0.123719,0.034310,0.012348,0.033495,0.020001,0.018725,0.125135,0.0
2,989.221774,24.026210,0.051743,0.033616,0.319908,0.001130,0.058497,0.132258,0.129040,0.035680,0.012915,0.034771,0.020145,0.021351,0.136430,0.0
3,750.704637,20.228831,0.061732,0.034126,0.531516,0.001165,0.068598,0.131361,0.128966,0.032824,0.013735,0.031436,0.020126,0.021869,0.113065,0.0
4,813.022177,21.395161,0.055015,0.033976,0.461716,0.001154,0.064392,0.131573,0.133104,0.034174,0.013168,0.031472,0.019780,0.020673,0.109533,0.0


In [8]:
X_train_df.to_csv('train_32.csv')

In [9]:
X_test = []
Y_test = []

for idx, texture_name in enumerate(classes):
    image_dir = os.path.join(test_dir, texture_name)
#     print(len(os.listdir(image_dir)))
    for image_name in os.listdir(image_dir):
        image = cv2.imread(os.path.join(image_dir, image_name))
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if 16 in PATCH_SIZES:
            for _ in range(NUM_PATCHES):
                w = np.random.randint(32-16)
                h = np.random.randint(32-16)

                image_gray_s = image_gray[w:w+PATCH_SIZE, h:h+PATCH_SIZE]

                glcm = greycomatrix(image_gray_s, distances=[6], angles=[0], levels=256,
                                   symmetric=False, normed=True)
                props = list()
                for prop in glcm_props:
                    props.append(greycoprops(glcm, f'{prop}')[0,0])

                X_test.append(props + laws_texture(image_p_gray))
                Y_test.append(idx)
        else:
            w = np.random.randint(32-16)
            h = np.random.randint(32-16)

            image_gray_s = image_gray[w:w+PATCH_SIZE, h:h+PATCH_SIZE]

            glcm = greycomatrix(image_gray_s, distances=[6], angles=[0], levels=256,
                               symmetric=False, normed=True)
            props = list()
            for prop in glcm_props:
                props.append(greycoprops(glcm, f'{prop}')[0,0])

            X_test.append(props + laws_texture(image_p_gray))
            Y_test.append(idx)

X_test = np.array(X_test)
Y_test = np.array(Y_test)
print('train data:', X_test.shape)
print('train label:', Y_test.shape)

train data: (250, 15)
train label: (250,)


In [10]:
test_data = np.append(X_test, Y_test.reshape(-1,1), axis=1)
test_df = pd.DataFrame(test_data, columns=['contrast', 'dissimilarity','homogeneity',
                                            'energy','correlation','ASM']+[f'TEM{i}' for i in range(9)]+['label'])

In [11]:
test_df.head()

,contrast,dissimilarity,homogeneity,energy,correlation,ASM,TEM0,TEM1,TEM2,TEM3,TEM4,TEM5,TEM6,TEM7,TEM8,label
0,517.888889,16.888889,0.059247,0.062854,0.368289,0.003951,0.074885,0.135045,0.145419,0.031347,0.014281,0.031341,0.021452,0.020227,0.107901,0.0
1,1696.858131,30.664360,0.041376,0.059229,0.417446,0.003508,0.074885,0.135045,0.145419,0.031347,0.014281,0.031341,0.021452,0.020227,0.107901,0.0
2,589.369565,19.157971,0.052798,0.041196,0.156381,0.001697,0.074885,0.135045,0.145419,0.031347,0.014281,0.031341,0.021452,0.020227,0.107901,0.0
3,767.377778,21.531852,0.051097,0.041243,0.080289,0.001701,0.074885,0.135045,0.145419,0.031347,0.014281,0.031341,0.021452,0.020227,0.107901,0.0
4,1460.014286,30.595238,0.025377,0.050057,0.263250,0.002506,0.074885,0.135045,0.145419,0.031347,0.014281,0.031341,0.021452,0.020227,0.107901,0.0


In [12]:
test_df.to_csv('test_32.csv')